In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
from numpy import asarray
import cv2
import matplotlib.pyplot as plt 

In [3]:
img =cv2.imread('../input/plant-pathology-2020-fgvc7/images/Test_0.jpg',cv2.IMREAD_GRAYSCALE)/255
plt.imshow(img)
plt.show()

In [4]:
img.shape

In [5]:
image_new=cv2.resize(img,(256,256))
image_new.shape

In [6]:
class Conv_op:
    def __init__(self,num_filters,filter_size):
        self.num_filters =num_filters#num filters for no.of kernels
        self.filter_size =filter_size# filter size for kernel size
        self.conv_filter=np.random.randn(num_filters,filter_size,filter_size)/(filter_size *filter_size)# normlising the kernels

    def image_region(self,image):#extracting patches
        height,width=image.shape
        self.image=image
        for j in range(height-self.filter_size+1):#size of the convoluted matrix
            for k in range(width-self.filter_size+1):
                image_patch=image[ j:(j+self.filter_size),k:(k+self.filter_size)]
                yield image_patch,j,k# different from normal func as it stores the buffer of output used when requiered

    def forword_prop(self,image):
        height,width=image.shape
        conv_out=np.zeros(shape=(height-self.filter_size+1,width-self.filter_size+1,self.num_filters))
        for image_patch,i,j in self.image_region(image):
            conv_out[i,j]=np.sum(image_patch*self.conv_filter,axis=(1,2))# mulitplying the patches and kernels for forward prop
        return conv_out

        
    def back_prop(self,dL_dout,learning_rate):#dl_dout from maxpool back prop
        dL_dF_params=np.zeros(self.conv_filter.shape)
        for image_patch,i,j in self.image_region(self.image):
            for k in range(self.num_filters):
                dL_dF_params[k]+=image_patch*dL_dout[i,j,k]

        self.conv_filter-=learning_rate*dL_dF_params#updating kernles
        return dL_dF_params

In [7]:
conn=Conv_op(18,7)
out=conn.forword_prop(image_new)
out.shape

In [8]:
plt.imshow(out[:,:,7])
plt.show

In [9]:


#reducing the image size by filter size and precerving information more accurately
class Max_pool:
    def __init__(self,filter_size):
        self.filter_size=filter_size
    
    def image_region(self,image):
        new_height=(image.shape[0]//self.filter_size)
        new_width=(image.shape[1]//self.filter_size)
        self.image=image

        for i in range(new_height):
            for j in range(new_width):
                image_patch=image[(i*self.filter_size):(i*self.filter_size+self.filter_size),(j*self.filter_size):(j*self.filter_size+self.filter_size)]
                yield image_patch,i,j
    def forward_prop(self,image):
        height,width,num_filters=image.shape
        output =np.zeros((height//self.filter_size,width//self.filter_size,num_filters))

        for image_patch,i,j in self.image_region(image):
            output[i,j]=np.amax(image_patch,axis=(0,1))
        return output

    def back_prop(self,dL_dout):
        dL_dmax_pool=np.zeros(self.image.shape)
        for image_patch,i,j in self.image_region(self.image):
            height,width,num_filters=image_patch.shape
            maximum_val=np.amax(image_patch,axis=(0,1))

            for i1 in range(height):
                for j1 in range(width):
                    for k1 in range(num_filters):
                        if image_patch[i1,j1,k1]==maximum_val[k1]:
                            dL_dmax_pool[i*self.filter_size+i1,j*self.filter_size+j1,k1]=dL_dout[i,j,k1]
            return dL_dmax_pool


    

In [10]:
conn2=Max_pool(4)
out2=conn2.forward_prop(out)
out2.shape

In [11]:
plt.imshow(out2[:,:,16])
plt.show()

In [12]:



class Softmax:
    def __init__(self,input_node,softmax_node):
        self.weight=np.random.randn(input_node,softmax_node)/input_node
        self.bias=np.zeros(softmax_node)

    def forward_prop(self,image):# flatteing the cube to one d aray
        self.orig_im_shape=image.shape
        image_modified=image.flatten()
        self.modified_input=image_modified
        output_val=np.dot(image_modified,self.weight)+self.bias
        self.out=output_val
        exp_out=np.exp(output_val)
        return exp_out/np.sum(exp_out,axis=0)

    def back_prop(self,dL_dout,learning_rates):
        for i,grad in enumerate(dL_dout):
            if grad==0:
                continue
            trasformation_eg=np.exp(self.out)
            s_total=np.sum(trasformation_eg)

            dy_dz=-trasformation_eg[i]*trasformation_eg/(s_total **2)
            dy_dz[i]=trasformation_eg[i]*(s_total-trasformation_eg[i])/(s_total **2)

            dz_dw=self.modified_input
            dz_db=1
            dz_d_inp=self.weight

            dL_dz=grad*dy_dz

            dL_dw=dz_dw[np.newaxis].T @ dL_dz[np.newaxis]
            dL_db=dL_dz*dz_db
            dL_d_inp=dz_d_inp @ dL_dz
        self.weight-=learning_rates*dL_dw
        self.bias-=learning_rates*dL_db

        return dL_d_inp.reshape(self.orig_im_shape)
        





In [13]:
conn3=Softmax(62*62*18,2)
out3=conn3.forward_prop(out2)
print(out3)# these are random values

In [14]:
image_dir="../input/plant-pathology-2020-fgvc7/images"

In [15]:
def normal_set(df,image_dir):
    images=[]#for storing images
    targets=[]#for keeping targtes
    for index,row in df.iterrows():
        image_id=row["image_id"]
        image_path=os.path.join(image_dir,image_id)
        image_path=image_path+".jpg"
        image=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)/255
        image=cv2.resize(image,(256,256))
        
        images.append(image)
        targets.append(int(row['healthy']))
    images=np.array(images)
    return images,targets

In [16]:
df=pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
df.head()

In [17]:
df['kfold']=-1
df=df.sample(frac=1).reset_index(drop=True)# resampling the elements for better model training
y=df.healthy.values

In [18]:
from sklearn import model_selection
kf=model_selection.StratifiedKFold(n_splits=int(1+np.log2(len(df))))# struges rule for no.of splits
for f, (t_,v_) in enumerate(kf.split(X=df,y=y)):
    df.loc[v_,'kfold']=f

In [19]:
df.kfold.value_counts()

In [20]:
train_1=df[df.kfold==5].reset_index(drop=True)
brgithness=2
x_train1,y_train1=normal_set(train_1,image_dir)

In [21]:
x_train1[0].shape

In [22]:
y_train1[9]

In [23]:
plt.imshow(x_train1[0])
plt.show()

In [24]:
conv=Conv_op(18,7)# first is no.of filtrers,second is size of filters ,256*256*1->250*250**18
pool=Max_pool(4)# dividing every dimension by 4  ,250*250*18->62*62*18
softmax=Softmax(62*62*18,2)# flattening them into single array  ,62*62*18

In [25]:
def cnn_forward_prop(image,label):
    out_p=conv.forword_prop(image)
    out_p=pool.forward_prop(out_p)
    out_p=softmax.forward_prop(out_p)
    cross_ent_loss=-np.log(out_p[label])
    accuracy_eval= 1 if np.argmax(out_p)==label else 0# keeping class labels
    return out_p,cross_ent_loss,accuracy_eval

In [26]:
# backprop by training cnn
def training(image,label,learn_rate=0.01):
    
    out,loss,acc=cnn_forward_prop(image,label)
    #start gradient
    gradient=np.zeros(10)
    gradient[label]=-1/out[label]
    
    #updating gradient by back prop
    grad_back=softmax.back_prop(gradient,learn_rate)
    grad_back=pool.back_prop(grad_back)
    grad_back=conv.back_prop(grad_back,learn_rate)
    
    return loss,acc

In [27]:
for epoch_n in range(1):
    #training cnn
    loss=0# making it batch stochastic process by keeping 32 as batch size
    num_correct=0
    for i,(im,label) in enumerate(zip(x_train1,y_train1)):
        if i%100==0:
            print(' stcohastic process of 100 steps:loss  and accuracy ',(i+1,loss/100,num_correct))
            loss=0
            num_correct=0
        l1,accu=training(im,label)
        loss+=l1
        num_correct+=accu

In [28]:
test=df[df.kfold==7].reset_index(drop=True)
x_test,y_test=normal_set(test,image_dir)

In [29]:
print('testing under progress')
loss=0
num_correct=0
for im,label in zip(x_test,y_test):
    m,l1,accu=cnn_forward_prop(im,label)
    loss+=l1
    num_correct+=accu
    num_test=len(x_test)
print('accuracy',num_correct/num_test)
print("test loss",loss/num_test)

improvements to the above model can be done by increasing the epochs.
increasing train dataset
in train dataset a mix of images whose images are bright or dark and flipped images for better training so as to overcome overfitting.